**test**

In [25]:
import plotly.plotly as plot
import pandas as pd

    Make some fake expression data
    

In [26]:
expression = [
    {"name" : "head", "expr" : -1},
    {"name" : "shoulders", "expr" : 2},
    {"name" : "knees", "expr" : 4},
    {"name" : "toes", "expr": -2}]

Pop it in a dataframe

In [27]:
df = pd.DataFrame(data=expression)

Let's see what the dataframe looks like...

In [28]:
df

,expr,name
0,-1,head
1,2,shoulders
2,4,knees
3,-2,toes


Ok, let's try to put this in a chart. 

In [29]:
from plotly.graph_objs import *

data = [Bar(x=df.name,
            y=df.expr)]

plot.iplot(data, filename='basic_bar')